In [1]:
import util

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
descriptions, images = util.make_descriptions_and_images()

In [4]:
labels, classes, class_to_index = util.make_labels_and_classes(descriptions)

In [ ]:
true_masks = np.zeros((size, 512, 640), np.uint8)
for i, descr in enumerate(descriptions):
  for shape in descr['shapes']:
    cv2.fillPoly(true_masks[i], [np.array(shape['points'])], 1)
#true_masks = true_masks[:, None, :, :]

In [ ]:
n_classes = 4
X, y = np.empty((n_classes, 0)).tolist(), np.empty((n_classes, 0)).tolist()
for class_, indexes in labels.items():
  current_class = class_to_index[class_]
  if (current_class > 2):
    continue
  for index in indexes:
    current_class = class_to_index[class_]
    if current_class == 0:
      if images[index].mean(axis=(0, 1))[1] >= 150:
        current_class = 3
    X[current_class].append(images[index])
    y[current_class].append(true_masks[index])
X = list(map(np.array, X))
y = list(map(np.array, y))
X[0].shape, X[1].shape, X[2].shape, X[3].shape

In [ ]:
mins, diffs = [[], [], [], [], [], []], [[], [], [], [], [], []]
for i in range(n_classes):
  min_, max_ = X[i].min(axis=0), X[i].max(axis=0)
  diff = max_ - min_
  del max_
  mins[0].append(min_.transpose(2, 0, 1).astype(np.uint8))
  diffs[0].append(diff.transpose(2, 0, 1).astype(np.uint8))
  min_im = Image.fromarray(min_)
  diff_im = Image.fromarray(diff)
  for j in range(1, 4):
    mins[j].append(np.asarray(min_im.rotate(90 * j, expand=True)).transpose(2, 0, 1).astype(np.uint8))
    diffs[j].append(np.asarray(diff_im.rotate(90 * j, expand=True)).transpose(2, 0, 1).astype(np.uint8))
  mins[4].append(np.asarray(min_im.transpose(Image.FLIP_TOP_BOTTOM)).transpose(2, 0, 1))
  diffs[4].append(np.asarray(diff_im.transpose(Image.FLIP_TOP_BOTTOM)).transpose(2, 0, 1))
  mins[5].append(np.asarray(min_im.transpose(Image.FLIP_LEFT_RIGHT)).transpose(2, 0, 1))
  diffs[5].append(np.asarray(diff_im.transpose(Image.FLIP_LEFT_RIGHT)).transpose(2, 0, 1))
for i in range(n_classes):
  for j in range(6):
    diffs[j][i] = np.where((diffs[j][i] == 0), np.inf, diffs[j][i])
def scale(X, class_, rotate):
  return 2 * (X - mins[rotate][class_]) / diffs[rotate][class_] - 1

In [ ]:
mins_save = np.array(mins[0])
diffs_save = np.array(diffs[0])
np.save('/content/drive/My Drive/ml_contests/digital_breakthrough/first4_mins.npy', mins_save)
np.save('/content/drive/My Drive/ml_contests/digital_breakthrough/first4_diffs.npy', diffs_save)

In [ ]:
#X_train, X_test, y_train, y_test = np.empty((n_classes, 0)).tolist(), np.empty((n_classes, 0)).tolist(),\
#                                   np.empty((n_classes, 0)).tolist(), np.empty((n_classes, 0)).tolist()
X_train_rotation = np.empty((n_classes, 0)).tolist()
X_train_rotation_rot = np.empty((n_classes, 0)).tolist()
X_train_aug, y_train_aug = np.empty((n_classes, 0)).tolist(), np.empty((n_classes, 0)).tolist()
X_train_aug_rot, y_train_aug_rot = np.empty((n_classes, 0)).tolist(), np.empty((n_classes, 0)).tolist()
for i in range(n_classes):
  #X_train[i], X_test[i], y_train[i], y_test[i] = map(np.ndarray.tolist, train_test_split(X[i], y[i], test_size=0.33))
  len_ = len(X[i])
  for j in range(len_):
    im = Image.fromarray(np.array(X[i][j], dtype=np.uint8))
    X_train_aug[i].append(np.asarray(im).transpose(2, 0, 1))
    im2 = im.rotate(90, expand=True)
    X_train_aug_rot[i].append(np.asarray(im2).transpose(2, 0, 1))
    im3 = im2.rotate(90, expand=True)
    X_train_aug[i].append(np.asarray(im3).transpose(2, 0, 1))
    im4 = im3.rotate(90, expand=True)
    X_train_aug_rot[i].append(np.asarray(im4).transpose(2, 0, 1))
    
    X_train_aug[i].append(np.asarray(im.transpose(Image.FLIP_TOP_BOTTOM)).transpose(2, 0, 1))
    X_train_aug[i].append(np.asarray(im.transpose(Image.FLIP_LEFT_RIGHT)).transpose(2, 0, 1))
    X_train_rotation[i] += [0, 2, 4, 5]
    X_train_rotation_rot[i] += [1, 3]
  for j in range(len_):
    im = Image.fromarray(np.array(y[i][j], dtype=np.uint8))
    y_train_aug[i].append(np.asarray(im)[None, :, :])
    im2 = im.rotate(90, expand=True)
    y_train_aug_rot[i].append(np.asarray(im2)[None, :, :])
    im3 = im2.rotate(90, expand=True)
    y_train_aug[i].append(np.asarray(im3)[None, :, :])
    im4 = im3.rotate(90, expand=True)
    y_train_aug_rot[i].append(np.asarray(im4)[None, :, :])
    
    y_train_aug[i].append(np.asarray(im.transpose(Image.FLIP_TOP_BOTTOM))[None, :, :])
    y_train_aug[i].append(np.asarray(im.transpose(Image.FLIP_LEFT_RIGHT))[None, :, :])
#X_train = list(map(np.array, X_train))
#X_test = list(map(np.array, X_test))
#del X_train, y_train

In [ ]:
np.array(X_train_aug[0]).shape, np.array(X_train_aug[1]).shape, np.array(X_train_aug[2]).shape, np.array(X_train_aug[3]).shape

((136, 3, 512, 640), (372, 3, 512, 640), (56, 3, 512, 640), (144, 3, 512, 640))

In [ ]:
y_test = list(map(np.array, y_test))
for i in range(n_classes):
  y_test[i] = y_test[i][:, None, :, :]
X_test = list(map(np.array, X_test))
for i in range(n_classes):
  X_test[i] = X_test[i].transpose(0, 3, 1, 2)

In [ ]:
class dataset(data.Dataset):
  def __init__(self, data, y, class_):
    self.rotation = data[1]
    self.len = len(data[0])
    self.x = data[0]
    self.y = y
    self.class_ = class_
    
  def __len__(self):
    return self.len

  def __getitem__(self, id):
    #print(self.class_, ' ', id, ' ', self.rotation[id])
    return (scale(self.x[id], self.class_, self.rotation[id]), self.y[id])

In [ ]:
train_datas, test_datas = np.empty((2 * n_classes, 0)).tolist(), np.empty((n_classes, 0)).tolist()
for i in range(n_classes):
  train_dataset = dataset((X_train_aug[i], X_train_rotation[i]), y_train_aug[i], i)
  train_dataset_rot = dataset((X_train_aug_rot[i], X_train_rotation_rot[i]), y_train_aug_rot[i], i)
  #test_dataset = dataset((X_test[i], np.zeros(X_test[i].shape[0], dtype=int)), y_test[i], i)
  batch_size = 4

  train_datas[i] = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  train_datas[n_classes + i] = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  #test_datas[i] = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
max_size = 512
class Unet(nn.Module):
  def __init__(self): # 512, 512
    super(Unet, self).__init__()
    self.conv1 = nn.Conv2d(3, max_size // 16, 3, padding=1) 
    self.conv2 = nn.Conv2d(max_size // 16, max_size // 16, 3, padding=1)
    self.conv3 = nn.Conv2d(max_size // 16, max_size // 8, 3, padding=1)
    self.conv4 = nn.Conv2d(max_size // 8, max_size // 8, 3, padding=1)
    self.conv5 = nn.Conv2d(max_size // 8, max_size // 4, 3, padding=1)
    self.conv6 = nn.Conv2d(max_size // 4, max_size // 4, 3, padding=1)
    self.conv7 = nn.Conv2d(max_size // 4, max_size // 2, 3, padding=1)
    self.conv8 = nn.Conv2d(max_size // 2, max_size // 2, 3, padding=1)
    self.conv9 = nn.Conv2d(max_size // 2, max_size, 3, padding=1)
    self.conv10 = nn.Conv2d(max_size, max_size, 3, padding=1)
    
    self.upconv1 = nn.ConvTranspose2d(max_size, max_size // 2, 2, 2)
    self.conv11 = nn.Conv2d(max_size, max_size // 2, 3, padding=1)
    self.conv12 = nn.Conv2d(max_size // 2, max_size // 2, 3, padding=1)

    self.upconv2 = nn.ConvTranspose2d(max_size // 2, max_size // 4, 2, 2)
    self.conv13 = nn.Conv2d(max_size // 2, max_size // 4, 3, padding=1)
    self.conv14 = nn.Conv2d(max_size // 4, max_size // 4, 3, padding=1)

    self.upconv3 = nn.ConvTranspose2d(max_size // 4, max_size // 8, 2, 2)
    self.conv15 = nn.Conv2d(max_size // 4, max_size // 8, 3, padding=1)
    self.conv16 = nn.Conv2d(max_size // 8, max_size // 8, 3, padding=1)

    self.upconv4 = nn.ConvTranspose2d(max_size // 8, max_size // 16, 2, 2)
    self.conv17 = nn.Conv2d(max_size // 8, max_size // 16, 3, padding=1)
    self.conv18 = nn.Conv2d(max_size // 16, max_size // 16, 3, padding=1)

    self.conv19 = nn.Conv2d(max_size // 16, 2, 1)
    self.a = nn.ReLU()
    self.pool = nn.MaxPool2d(2, 2)

  def forward(self, x): # 512, 640 
    c1 = self.a(self.conv2(self.a(self.conv1(x)))) # 8 512 640 <- 8 512 640 <- 3 512 640
    p1 = self.pool(c1) # 8 256, 320
    c2 = self.a(self.conv4(self.a(self.conv3(p1)))) # 16 256 320 <- 16 256 320 <- 8 256 320
    p2 = self.pool(c2) # 128, 160
    c3 = self.a(self.conv6(self.a(self.conv5(p2)))) # 32 128 160 <- 32 128 160 <- 16 128 160
    p3 = self.pool(c3) # 64, 80
    c4 = self.a(self.conv8(self.a(self.conv7(p3)))) # 64 64 80 <- 64 64 80 <- 32 64 80
    p4 = self.pool(c4) # 32, 40
    c5 = self.a(self.conv10(self.a(self.conv9(p4)))) # 128 32 40 <- 128 32 40 <- 64 32 40
    #print(c4.shape, ' ', c5.shape, ' ', self.upconv1(c5).shape)
    u6 = torch.cat((self.upconv1(c5), c4), dim=1)  # 128 64 80 <- 64 64 80 <- 128 32 40
    c6 = self.a(self.conv12(self.a(self.conv11(u6)))) # 64 64 80 <- 64 64 80 <- 128 64 80
    u7 = torch.cat((self.upconv2(c6), c3), dim=1) # 64 128 160 <- 32 128 160 <- 64 64 80
    c7 = self.a(self.conv14(self.a(self.conv13(u7)))) # 32 128 160 <- 32 128 160 <- 64 128 160
    u8 = torch.cat((self.upconv3(c7), c2), dim=1) # 32 256 320 <- 16 256 320 <- 32 128 160
    c8 = self.a(self.conv16(self.a(self.conv15(u8)))) # 16 256 320 <- 16 256 320 <- 32 256 320
    u9 = torch.cat((self.upconv4(c8), c1), dim=1) # 16 512 640 <- 8 512 640 <- 16 256 320
    c9 = self.a(self.conv18(self.a(self.conv17(u9)))) # 8 512 640 <- 8 512 640 <- 16 512 640

    return self.conv19(c9) # 2 512 640 <- 8 512 640

In [ ]:
def dice_loss(x, y):
  return (1 - (2 * (x * y).sum(dim=(2, 3)) / (x.sum(dim=(2, 3)) + y.sum(dim=(2, 3)))).mean(dim=(0, 1)))

In [ ]:
device = torch.device("cuda:0")

In [ ]:
loss1 = nn.CrossEntropyLoss()
loss2 = dice_loss
u_models = np.empty((n_classes, 0)).tolist()
optims = np.empty((n_classes, 0)).tolist()
for i in range(n_classes):
  u_models[i] = Unet().double().to(device)
  optims[i] = torch.optim.Adam(u_models[i].parameters(), lr=0.0005)

In [ ]:
active = nn.Softmax2d()
for epoch in range(20):
    train_batch_loss = [[], [], [], []]
    test_batch_loss = [[], [], [], []]
    train_batch_score = [[], [], [], []]
    test_batch_score = [[], [], [], []]
    start_epoch = time.time()
    for i, train_data in enumerate(train_datas):
      if i >= n_classes:
        i -= n_classes
      u_models[i].train()
      for batch in train_data:
        start_batch = time.time()
        optims[i].zero_grad()
        scores = u_models[i](batch[0].to(device=device))
        predections = scores.argmax(dim=1).cpu().detach().numpy()
        batch[1] = batch[1].long().to(device=device)
        groud_truth = batch[1].squeeze(dim=1)
        current_loss = loss1(scores, groud_truth) + loss2(active(scores), torch.cat((batch[1].logical_not().long(), batch[1]), dim=1))
        current_loss.backward()
        optims[i].step()
        train_batch_loss[i].append(current_loss.item())
        groud_truth = groud_truth.cpu().detach().numpy()
        train_batch_score[i] += (np.count_nonzero(np.logical_and(groud_truth, predections), axis=(1, 2)) / 
                                    np.count_nonzero(np.logical_or(groud_truth, predections), axis=(1, 2))).tolist()
    
    
    for i, test_data in enumerate(test_datas):
      u_models[i].eval()
      for batch in test_data:
        scores = u_models[i](batch[0].to(device=device))
        predections = scores.argmax(dim=1).cpu().detach().numpy()
        batch[1] = batch[1].long().to(device=device)
        groud_truth = batch[1].squeeze(dim=1)
        current_loss = loss1(scores, batch[1].squeeze(dim=1)) + loss2(active(scores), torch.cat((batch[1].logical_not().long(), batch[1]), dim=1))
        test_batch_loss[i].append(current_loss.item())
        groud_truth = groud_truth.cpu().detach().numpy()
        test_batch_score[i] += (np.count_nonzero(np.logical_and(groud_truth, predections), axis=(1, 2)) / 
                                  np.count_nonzero(np.logical_or(groud_truth, predections), axis=(1, 2))).tolist()
    
    cetrl, cettl, trbs, ttbs = np.zeros(n_classes), np.zeros(n_classes), np.zeros(n_classes), np.zeros(n_classes), 
    for j in range(n_classes):
      cetrl[j] = np.mean(train_batch_loss[j])
      cettl[j] = np.mean(test_batch_loss[j]) 
      trbs[j] = np.mean(train_batch_score[j])
      ttbs[j] = np.mean(test_batch_score[j])
    print(epoch, " time took - ", time.time() - start_epoch, "\ntrain_loss = ", cetrl, "\ntest_loss = ", cettl, '\ntrain score = ', trbs, '\ntest score = ', ttbs)

/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


0  time took -  491.9342532157898 
train_loss =  [0.80722819 0.618451   1.04355644 0.945009  ] 
test_loss =  [0.62829842 0.37586302 0.56504842 0.7732091 ] 
train score =  [0.         0.0083714  0.00747949 0.00432936] 
test score =  [0.         0.22649061 0.         0.        ]
1  time took -  491.64195108413696 
train_loss =  [0.50810538 0.18317762 0.62764598 0.69430403] 
test_loss =  [0.29683626 0.12696378 0.57251429 0.61683554] 
train score =  [0.         0.59363788 0.         0.        ] 
test score =  [0.        0.6910495 0.        0.       ]
2  time took -  491.41602873802185 
train_loss =  [0.23365665 0.0997813  0.61243527 0.46575063] 
test_loss =  [0.0813315  0.09069327 0.55274237 0.36107215] 
train score =  [0.62823185 0.72089017 0.         0.31763101] 
test score =  [0.8012017  0.74026556 0.         0.58723301]
3  time took -  491.5036549568176 
train_loss =  [0.07578415 0.09054568 0.5869711  0.20623085] 
test_loss =  [0.07175322 0.0819265  0.52776877 0.16746264] 
train score 

In [ ]:
# 0 - 6, 1 - 12, 2 - 17, 3 - 16

In [ ]:
# bn+tanh only 4 last 20 epoches

0  time took -  334.73125195503235 
train_loss =  [1.27510557 1.12598527 1.34790749 1.14931956] 
test_loss =  [1.19712935 0.82487849 1.23548838 1.10489169] 
train score =  [0.01507985 0.02053684 0.00549052 0.02816047] 
test score =  [0.04048506 0.00316424 0.00757537 0.01362815]
1  time took -  334.51902437210083 
train_loss =  [1.08527119 0.74162772 1.06511032 0.84611308] 
test_loss =  [1.02027778 0.61248738 0.94202092 0.76838077] 
train score =  [0.00808443 0.08793022 0.00648866 0.01095671] 
test score =  [6.68355146e-04 2.04730066e-01 1.46419464e-04 9.53140193e-04]
2  time took -  334.73840260505676 
train_loss =  [0.90720696 0.55823327 0.82458723 0.66660186] 
test_loss =  [0.88650907 0.47268154 0.72958807 0.63495002] 
train score =  [0.01977812 0.35232923 0.0005051  0.13556608] 
test score =  [0.         0.27610694 0.         0.14955263]
3  time took -  334.49046564102173 
train_loss =  [0.79024566 0.42466221 0.69208882 0.57787752] 
test_loss =  [0.75054038 0.37120604 0.65338301 0.5

In [ ]:
# max size - 512, bs - 2, lr - 0.0005, bn + tanh

/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


0  time took -  900.8071782588959 
train_loss =  [0.88588502 0.79897586 1.23730298 1.13471253 1.23905224 1.07641844] 
test_loss =  [0.71257704 0.53063296 1.13609627 0.85824928 1.11546308 0.9864407 ] 
train score =  [0.30116513 0.34251821 0.0220032  0.0145953  0.005514   0.02405696] 
test score =  [0.15962902 0.53898676 0.05196306 0.         0.00681989 0.02429016]
1  time took -  900.2338981628418 
train_loss =  [0.43594538 0.35437245 1.04930878 0.74999283 1.03981589 0.8565874 ] 
test_loss =  [0.62026321 0.20292214 0.96537592 0.66814233 0.89537281 0.7788853 ] 
train score =  [0.53771895 0.61704724 0.02316369 0.0062796  0.00563939 0.01077256] 
test score =  [1.98044062e-01 7.00012249e-01 4.26233940e-05 2.57347405e-03
 0.00000000e+00 3.28064240e-03]
2  time took -  899.9886522293091 
train_loss =  [0.23253207 0.13937567 0.88113007 0.62846184 0.82098116 0.69228435] 
test_loss =  [0.47299188 0.1675915  0.81693655 0.58305039 0.72480642 0.68093775] 
train score =  [6.97137028e-01 7.26195168e-

In [ ]:
# max size - 512, lr - 0.0005, bs - 4, Unet with relu

0  time took -  721.6788597106934 
train_loss =  [0.6389396  0.59311313 1.12914431 0.71818613 0.82036042 0.84664015] 
test_loss =  [1.33673537 0.35569334 0.61198898 0.60306305 0.55575149 0.60353431] 
train score =  [0.1311722  0.00034535 0.00721091 0.         0.         0.00168007] 
test score =  [0.02926223 0.         0.         0.         0.         0.        ]
1  time took -  721.0491104125977 
train_loss =  [0.27287945 0.1754974  0.61857249 0.46952755 0.53219959 0.56517482] 
test_loss =  [3.37140069 0.11486308 0.60206729 0.50052358 0.53908552 0.56468886] 
train score =  [0.52796348 0.56558197 0.         0.16038902 0.         0.        ] 
test score =  [0.43217423 0.69087778 0.         0.3209337  0.         0.        ]
2  time took -  720.9163219928741 
train_loss =  [0.15841827 0.11271382 0.59817462 0.28714283 0.52224577 0.54215328] 
test_loss =  [0.67893372 0.10322325 0.64312099 0.38330916 0.53627023 0.51429862] 
train score =  [0.6848175  0.69273944 0.         0.42984783 0.      